In [2]:
import pandas as pd
from rapidfuzz import fuzz, distance
from collections import Counter
import re
from tqdm import tqdm
import os

# === 1. Load CSV ===
csv_file = "20250903_Extrait_Constatations_F2.csv"
df = pd.read_csv(csv_file, delimiter=';', dtype=str)
df.columns = df.columns.str.strip()
comment_col = "Commentaire"
df = df.dropna(subset=[comment_col])

# === 2. Text preprocessing helpers ===
def clean_words(text):
    if pd.isna(text):
        return []
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    words = text.split()
    words = [w for w in words if len(w) > 1]  # ignore 1-letter words
    return words

def generate_clusters(words, min_len=2, max_len=5):
    clusters = []
    for n in range(min_len, min(max_len+1, len(words)+1)):
        for i in range(len(words)-n+1):
            cluster = ' '.join(words[i:i+n])
            # skip clusters with all single-letter words
            if all(len(w) == 1 for w in cluster.split()):
                continue
            clusters.append(cluster)
    return clusters

# === 3. Extract clusters from Commentaire ===
print("Extracting clusters from Commentaire...")
all_clusters = []
for com in tqdm(df[comment_col], desc="Extracting"):
    words = clean_words(com)
    clusters = generate_clusters(words, min_len=2, max_len=5)
    all_clusters.extend(clusters)

# Count exact occurrences
cluster_counter = Counter(all_clusters)
cluster_candidates = {k:v for k,v in cluster_counter.items() if v >= 7}
print(f"→ Found {len(cluster_candidates)} unique clusters with ≥7 occurrences.")

# === 4. Fuzzy clustering parameters ===
threshold_similarity = 90  # %
max_typo_chars = 4         # edit distance threshold
checkpoint_every = 50
checkpoint_file = "checkpoint_commentaire_clusters.csv"

# === 5. Resume from checkpoint if exists ===
results = []
processed = set()
if os.path.exists(checkpoint_file):
    checkpoint_df = pd.read_csv(checkpoint_file, sep=';')
    processed = set(checkpoint_df["Cluster_Representative"].tolist())
    results = checkpoint_df.to_dict('records')
    print(f"Resuming from checkpoint → {len(processed)} clusters already processed.")

# === 6. Fuzzy clustering ===
cluster_groups = {}
candidates_list = list(cluster_candidates.keys())

print("Performing fuzzy clustering with RapidFuzz...")
for i, cluster in enumerate(tqdm(candidates_list, desc="Fuzzy clustering")):
    if cluster in processed:
        continue

    found = False
    for key in cluster_groups.keys():
        similarity = fuzz.ratio(cluster, key)
        edit_dist = distance.Levenshtein.distance(cluster, key)
        if similarity >= threshold_similarity or edit_dist <= max_typo_chars:
            cluster_groups[key].append(cluster)
            found = True
            break
    if not found:
        cluster_groups[cluster] = [cluster]

    # === Checkpoint saving ===
    if (i + 1) % checkpoint_every == 0:
        partial_results = []
        total_counts = sum(cluster_counter.values())
        for key, variants in cluster_groups.items():
            count = sum([cluster_counter[v] for v in variants])
            per_mille = round(count / total_counts * 1000, 2)
            partial_results.append({
                "Cluster_Representative": key,
                "Count": count,
                "Variations": '|'.join(variants),
                "PerMille": per_mille
            })
        pd.DataFrame(partial_results).to_csv(checkpoint_file, sep=';', index=False)
        print(f"Checkpoint saved at {i+1} clusters → {checkpoint_file}")

# === 7. Final aggregation ===
print("Finalizing results...")
total_counts = sum(cluster_counter.values())
results = []
for key, variants in tqdm(cluster_groups.items(), desc="Final aggregation"):
    count = sum([cluster_counter[v] for v in variants])
    per_mille = round(count / total_counts * 1000, 2)
    results.append({
        "Cluster_Representative": key,
        "Count": count,
        "Variations": '|'.join(variants),
        "PerMille": per_mille
    })

summary_df = pd.DataFrame(results).sort_values(by="Count", ascending=False)
summary_df.to_csv("commentaire_clusters_summary_final.csv", sep=';', index=False)
display(summary_df)
print("✅ Saved final results as 'commentaire_clusters_summary_final.csv'")


Extracting clusters from Commentaire...


Extracting: 100%|█████████████████████████████████████████████████████████████| 27665/27665 [00:00<00:00, 43328.58it/s]


→ Found 4671 unique clusters with ≥7 occurrences.
Performing fuzzy clustering with RapidFuzz...


Fuzzy clustering:   2%|█                                                            | 84/4671 [00:00<00:05, 827.27it/s]

Checkpoint saved at 50 clusters → checkpoint_commentaire_clusters.csv
Checkpoint saved at 100 clusters → checkpoint_commentaire_clusters.csv


...

Fuzzy clustering: 100%|████████████████████████████████████████████████████████████| 4671/4671 [02:28<00:00, 31.38it/s]


Finalizing results...


Final aggregation: 100%|███████████████████████████████████████████████████████| 1898/1898 [00:00<00:00, 120826.72it/s]


,Cluster_Representative,Count,Variations,PerMille
0,stk bis,6517,stk bis|stk bis 300|stk klein|stk bis 700|stk ...,19.42
1,bis 300,5626,bis 300|bis 300 cm2|bis cm|bis 700|bis 100|bis...,16.76
5,korrodierter bewehrung,5028,korrodierter bewehrung|mit korrodierter bewehr...,14.98
3,cm2 mit,4809,cm2 mit|che mit|700 cm2 mit|100 cm2 mit|500 cm...,14.33
4,mit korrodierter,4469,mit korrodierter|cm2 mit korrodierter|stk korr...,13.32
...,...,...,...,...
1893,querschnittsverlust die stabanker und flanschm...,7,querschnittsverlust die stabanker und flanschm...,0.02
1894,flanschmuttern weisen keinen korrosionsschutz ...,7,flanschmuttern weisen keinen korrosionsschutz ...,0.02
1895,weisen keinen korrosionsschutz verzinkung auf,7,weisen keinen korrosionsschutz verzinkung auf,0.02
1896,korrosionsschutz verzinkung auf derzeitige kor...,7,korrosionsschutz verzinkung auf derzeitige kor...,0.02


✅ Saved final results as 'commentaire_clusters_summary_final.csv'
